In [1]:
import nltk
nltk.download('stopwords')

import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from bs4 import BeautifulSoup 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.request
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

print('=3')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Choi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


=3


In [4]:

BASE_DIR = os.getcwd()
print(BASE_DIR)

#\돈버튼 아래
data_dir = BASE_DIR
data_dir += '\data'

data = pd.read_csv(data_dir+"/Reviews.csv", nrows=100000)
print('전체 샘플수 :', (len(data)))

c:\github\Simple_AI
전체 샘플수 : 100000


In [5]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [6]:
data = data[['Text','Summary']]
data.head()

#랜덤한 15개 샘플 출력
data.sample(15)

,Text,Summary
30186,I love this mix because it is so simple to mak...,Cooking for 5
28087,"Unlike the McDonald's oatmeal, which made head...","simple, convenient, and healthy"
46091,This is the only brand which I could find that...,Decaf Chai spiced loose tea
62592,Lily's bars are among the best chocolate I've ...,Wonderful chocolate!
98313,I always like to keep a box of energy bar in t...,Yummy!!!!
12788,unbelievable good taste - these anchovies are ...,wow
65009,I was a little hesitant about ordering these a...,Surprised pleasantly and amazed.
48433,I received this in my Inluenster Summer Vox Bo...,Workday delight!
82399,I just love this tea. It's the only tea I dri...,Bentley's Ginger Peach White Tea review
11741,"Eat your heart out , Godiva<br /><br />Be envi...",WIFE WENT LUNAR!
